## Homework #3

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

In [2]:
ca_housing = pd.read_csv('C:/Users/davej/.jupyter/mlzoomcamp/02-regression/ca_housing_prices.csv')
ca_housing = ca_housing[['latitude','longitude','housing_median_age','total_rooms','total_bedrooms','population',
                         'households','median_income','median_house_value', 'ocean_proximity']]

###  Question 1

In [3]:
ca_housing.isnull().sum()
ca_housing['total_bedrooms'] = ca_housing['total_bedrooms'].fillna(0)
ca_housing['rooms_per_household'] = ca_housing['total_rooms'] / ca_housing['households']
ca_housing['bedrooms_per_room'] = ca_housing['total_bedrooms'] / ca_housing['total_rooms']
ca_housing['population_per_household'] = ca_housing['population'] / ca_housing['households']
ca_housing['ocean_proximity'].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

###  Question 2

In [4]:
ca_housing.info()
numerical = ['latitude', 'longitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population',
             'households', 'median_income', 'median_house_value', 'rooms_per_household', 'bedrooms_per_room',
             'population_per_household']
ca_housing[numerical].corr()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   latitude                  20640 non-null  float64
 1   longitude                 20640 non-null  float64
 2   housing_median_age        20640 non-null  float64
 3   total_rooms               20640 non-null  float64
 4   total_bedrooms            20640 non-null  float64
 5   population                20640 non-null  float64
 6   households                20640 non-null  float64
 7   median_income             20640 non-null  float64
 8   median_house_value        20640 non-null  float64
 9   ocean_proximity           20640 non-null  object 
 10  rooms_per_household       20640 non-null  float64
 11  bedrooms_per_room         20640 non-null  float64
 12  population_per_household  20640 non-null  float64
dtypes: float64(12), object(1)
memory usage: 2.0+ MB


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,rooms_per_household,bedrooms_per_room,population_per_household
latitude,1.000000,-0.924664,0.011173,-0.036100,-0.065318,-0.108785,-0.071035,-0.079809,-0.144160,0.106389,-0.104112,0.002366
longitude,-0.924664,1.000000,-0.108197,0.044568,0.068082,0.099773,0.055310,-0.015176,-0.045967,-0.027540,0.084836,0.002476
housing_median_age,0.011173,-0.108197,1.000000,-0.361262,-0.317063,-0.296244,-0.302916,-0.119034,0.105623,-0.153277,0.125396,0.013191
total_rooms,-0.036100,0.044568,-0.361262,1.000000,0.920196,0.857126,0.918484,0.198050,0.134153,0.133798,-0.174583,-0.024581
total_bedrooms,-0.065318,0.068082,-0.317063,0.920196,1.000000,0.866266,0.966507,-0.007295,0.049148,0.002717,0.122205,-0.028019
population,-0.108785,0.099773,-0.296244,0.857126,0.866266,1.000000,0.907222,0.004834,-0.024650,-0.072213,0.031397,0.069863
households,-0.071035,0.055310,-0.302916,0.918484,0.966507,0.907222,1.000000,0.013033,0.065843,-0.080598,0.059818,-0.027309
median_income,-0.079809,-0.015176,-0.119034,0.198050,-0.007295,0.004834,0.013033,1.000000,0.688075,0.326895,-0.573836,0.018766
median_house_value,-0.144160,-0.045967,0.105623,0.134153,0.049148,-0.024650,0.065843,0.688075,1.000000,0.151948,-0.238759,-0.023737
rooms_per_household,0.106389,-0.027540,-0.153277,0.133798,0.002717,-0.072213,-0.080598,0.326895,0.151948,1.000000,-0.387465,-0.004852


###  Question 3

In [5]:
ca_housing['above_average'] = (ca_housing['median_house_value'] > ca_housing['median_house_value'].mean()).astype(int)

In [6]:
#setting up the validation framework using sklearn
cah_full_train, cah_test = train_test_split(ca_housing, test_size=0.2, random_state=42)
len(cah_full_train), len(cah_test)
cah_train, cah_val = train_test_split(cah_full_train, test_size=0.25, random_state=42)
len(cah_train), len(cah_val)

(12384, 4128)

In [7]:
#reset indices for each data subset
cah_train = cah_train.reset_index(drop=True)
cah_val = cah_val.reset_index(drop=True)
cah_test = cah_test.reset_index(drop=True)
cah_full_train = cah_full_train.reset_index(drop=True)

In [8]:
#creating vector for y vars
y_train = cah_train['above_average'].values
y_val = cah_val['above_average'].values
y_test = cah_test['above_average'].values
y_full_train = cah_full_train['above_average'].values

In [9]:
#delete median_house_value vars from subsets so that we don't accidentally use it as a feature to predict target var
cah_train = cah_train.drop(['median_house_value'], axis=1)
cah_val = cah_val.drop(['median_house_value'], axis=1)
cah_test = cah_test.drop(['median_house_value'], axis=1)

In [10]:
round(mutual_info_score(cah_train['above_average'], cah_train['ocean_proximity']),2)

0.1

###  Question 4

In [11]:
#training data
train_columns = numerical.copy()
train_columns.remove('median_house_value')
train_columns = train_columns + ['ocean_proximity']
train_dicts = cah_train[train_columns].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
val_dicts = cah_val[train_columns].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
#training logistic regression
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:, 1]
churn_decision = (y_pred >= 0.5)
churn_decision = churn_decision.astype(int)
global_score = (y_val == churn_decision).mean()
global_score

0.8359980620155039

###  Question 5

In [13]:
#calculate model accuracy for each iteration that excludes a different feature from the training set
for var in train_columns:
    tc_copy = train_columns.copy()
    tc_copy.remove(var)
    train_dicts = cah_train[tc_copy].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(train_dicts)
    val_dicts = cah_val[tc_copy].to_dict(orient='records')
    X_val = dv.transform(val_dicts)

    model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    churn_decision = (y_pred >= 0.5)
    churn_decision = churn_decision.astype(int)
    score = (y_val == churn_decision).mean()
    print(var, global_score - score)

latitude 0.003633720930232509
longitude 0.004118217054263518
housing_median_age 0.0043604651162790775
total_rooms -0.00024224806201555982
total_bedrooms -0.001211240310077577
population 0.009689922480620172
households 0.0019379844961240345
median_income 0.050629844961240345
rooms_per_household 0.0007267441860464574
bedrooms_per_room -0.00024224806201555982
population_per_household 0.0002422480620154488
ocean_proximity 0.015746124031007724


###  Question 6

In [14]:
#setting up the validation framework using sklearn
cah_full_train, cah_test = train_test_split(ca_housing, test_size=0.2, random_state=42)
len(cah_full_train), len(cah_test)
cah_train, cah_val = train_test_split(cah_full_train, test_size=0.25, random_state=42)
len(cah_train), len(cah_val)

(12384, 4128)

In [15]:
#reset indices for each data subset
cah_train = cah_train.reset_index(drop=True)
cah_val = cah_val.reset_index(drop=True)

In [16]:
#creating vector for y vars
y_train_q6 =  np.log1p(cah_train['median_house_value'].values)
y_val_q6 =  np.log1p(cah_val['median_house_value'].values)

In [17]:
#delete median_house_value vars from subsets so that we don't accidentally use it as a feature to predict target var
cah_train = cah_train.drop(['median_house_value'], axis=1)
cah_val = cah_val.drop(['median_house_value'], axis=1)

In [18]:
#training data
train_dicts = cah_train[train_columns].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
val_dicts = cah_val[train_columns].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [19]:
#set up rmse calculation function
def rmse(y,y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [20]:
#training ridge for different alpha values
for a in [0.00, 0.01, 0.10, 1.00, 10.0]:
    model = Ridge(alpha=a, solver="sag", random_state=42)
    model.fit(X_train, y_train_q6)
    y_pred = model.predict(X_val)
    score = round(rmse(y_val_q6, y_pred),3)
    print(a, score)

0.0 0.524
0.01 0.524
0.1 0.524
1.0 0.524
10.0 0.524
